## PyTorch Implementation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim
from torch.utils.data import DataLoader , TensorDataset
import tensorflow as tf
from   torch.optim.lr_scheduler import ExponentialLR as ExponentialLR

In [2]:
import pandas as pd
data = pd.read_csv('Bank_Personal_Loan_Modelling.csv')

In [3]:
data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [4]:
data.drop(['ID'] , inplace = True , axis = 1)

In [5]:
x = data.drop(['Personal Loan'] , axis = 1).values
y = data['Personal Loan'].values

In [6]:
x = torch.tensor(x , dtype = torch.float64)
y = torch.tensor(y)
y = y.type(torch.LongTensor)

In [7]:
class NN(nn.Module):
    def __init__(self , input_size , hidden1 , hidden2 ,  output_size):
        super().__init__()
        self.input_ = nn.Linear(input_size , hidden1)
        self.hidden1_ = nn.Linear(hidden1 , hidden2)
        self.out = nn.Linear(hidden2 , output_size)
    def forward(self , x):
        x = f.relu(self.input_(x))
        x = torch.sigmoid(self.hidden1_(x))
        x = self.out(x)
        x = f.sigmoid(x)
        return x

In [8]:
model = NN(12 , 10 , 8 , 2)

In [9]:
model.parameters

<bound method Module.parameters of NN(
  (input_): Linear(in_features=12, out_features=10, bias=True)
  (hidden1_): Linear(in_features=10, out_features=8, bias=True)
  (out): Linear(in_features=8, out_features=2, bias=True)
)>

In [10]:
loss_function = nn.CrossEntropyLoss()

In [11]:
optimizer = torch.optim.Adam( model.parameters(), lr=100)
scheduler = ExponentialLR(optimizer, gamma=0.9)

In [12]:
n_epochs = 1000

In [13]:
data = TensorDataset(x , y)
data = DataLoader(data)

In [14]:
with tf.device('/gpu:0'):
    final_losses = []
    for i in range(n_epochs):
        y_pred = model(x.float())
        loss = loss_function(y_pred , y)
        final_losses.append(loss.item())
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        if (i%10 == 0):
            print("Epoch {} has loss of {}".format(i , loss.item()))

Epoch 0 has loss of 0.6916993856430054
Epoch 10 has loss of 0.409261554479599
Epoch 20 has loss of 0.409261554479599
Epoch 30 has loss of 0.409261554479599
Epoch 40 has loss of 0.409261554479599
Epoch 50 has loss of 0.409261554479599
Epoch 60 has loss of 0.409261554479599
Epoch 70 has loss of 0.409261554479599
Epoch 80 has loss of 0.409261554479599
Epoch 90 has loss of 0.409261554479599
Epoch 100 has loss of 0.409261554479599
Epoch 110 has loss of 0.409261554479599
Epoch 120 has loss of 0.409261554479599
Epoch 130 has loss of 0.409261554479599
Epoch 140 has loss of 0.409261554479599
Epoch 150 has loss of 0.409261554479599
Epoch 160 has loss of 0.409261554479599
Epoch 170 has loss of 0.409261554479599
Epoch 180 has loss of 0.409261554479599
Epoch 190 has loss of 0.409261554479599
Epoch 200 has loss of 0.409261554479599
Epoch 210 has loss of 0.409261554479599
Epoch 220 has loss of 0.409261554479599
Epoch 230 has loss of 0.409261554479599
Epoch 240 has loss of 0.409261554479599
Epoch 250 

In [15]:
predictions = []
with torch.no_grad():
    for i , data in enumerate(x):
        y_pred = (model(data.float()))
        predictions.append(y_pred.argmax().item())

In [16]:
from sklearn.metrics import classification_report

In [17]:
print(classification_report(y , predictions))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      4520
           1       0.00      0.00      0.00       480

    accuracy                           0.90      5000
   macro avg       0.45      0.50      0.47      5000
weighted avg       0.82      0.90      0.86      5000



C:\Users\Srinidhi Tarigoppula\OneDrive\Documents\New folder\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Srinidhi Tarigoppula\OneDrive\Documents\New folder\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Srinidhi Tarigoppula\OneDrive\Documents\New folder\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa